# The Cost of Python Built-ins

## 1. Introduction

Built-in data structures and their operations in Python come with implicit performance costs.  

Understanding these costs — primarily in terms of **time complexity** — helps write more efficient, scalable code.  

Below is a reference overview of common Python built-ins: lists, dictionaries, sets, and some of their typical operations.

---

## 2. Lists (`list`)

Python `list` is implemented as a dynamic array: resizing and memory allocation happen behind the scenes.

| Operation | Example | Average Case | Worst/Amortized Case |
|-----------|---------|--------------|----------------------|
| Index access / assignment | `lst[i]`, `lst[i] = v` | O(1) | O(1) |
| `append(item)` | `lst.append(x)` | O(1) amortized | Occasionally O(n) when resizing — but amortized cost is O(1) |
| `pop()` from end | `lst.pop()` | O(1) | O(1) |
| Insert/remove at arbitrary index / `pop(i)` / `remove(item)` | `lst.insert(...)`, `lst.pop(i)`, `lst.remove(x)` | O(n) | O(n) |
| Containment test (`x in lst`) | `x in lst` | O(n) | O(n) |
| Iteration / slicing / copy / length / equality / concatenation / sort | many operations | typically O(n) or worse (e.g. O(n log n) for sort) | same order |

!!! Warning :

- pop(0) (Start of list): When you remove the first item, you leave a "hole" at index 0. Python cannot have a hole at the start of an array. It must shift every single remaining item one step to the left to fill the gap.

Cost: $O(n)$ (Linear Time).

## Using `collections.deque` for Efficient Double-Ended Operations

When you need to efficiently add or remove elements from **both ends** of a sequence, Python offers the `deque` (Double-Ended Queue).

---

### 1. What is a `deque`?

A `deque` is a data structure implemented in Python as a **doubly linked list of blocks**.  
Each block holds a group of elements and points to the next block.

This structure allows efficient operations at both ends of the sequence.

---

### 2. Why `deque` is Efficient

#### `popleft()`
- Removes the first element by simply unlinking the first block/node.
- No shifting of other elements.
- **Cost:**  
  \[
  O(1)
  \]
  Constant-time operation.

#### `appendleft()`  
- Adds an element to the front in constant time.
- Also avoids shifting elements, unlike lists.

---

### 3. Summary Table

| Operation        | List (`list`) | Deque (`deque`) |
|------------------|---------------|------------------|
| `append(x)`      | O(1)          | O(1)             |
| `appendleft(x)`  | O(n)          | O(1)             |
| `popleft()`      | O(n)          | O(1)             |
| `pop()`          | O(1)          | O(1)             |

---

### 4. Recommended Use Case

Use a `deque` when:
- You need queue or double-ended queue behavior.

- You frequently add/remove elements from the **front** of the sequence.

## 3. Dictionaries (`dict`) and Sets (`set`)

Python `dict` and `set` are implemented using hash tables, enabling efficient key lookups, insertions, and deletions in average constant time.

#### Dictionary: Common Operations

| Operation | Example | Average Case | Worst Case |
|-----------|---------|--------------|------------|
| Lookup / get / membership test | `d[key]`, `d.get(key)`, `key in d` | O(1) | O(n) in case of severe hash collisions |
| Insertion / update | `d[key] = val` | O(1) | O(n) (rare, due to rehash/resizing) |
| Deletion | `del d[key]` or `d.pop(key)` | O(1) | O(n) (rare)  |
| Iteration (keys, values, items) | `for k in d:` | O(n) | O(n) |
| Copy / `dict.copy()` | — | O(n) | O(n) |

Because of hashing, dictionary and set operations are often much faster than equivalent operations on lists — especially for membership checks and lookups.

#### Set: Basic behavior (similar to `dict` for membership and insertion)

- Membership test `x in set`: average-case O(1), worst-case O(n) if many hash collisions.

- `add(item)`, `remove(item)`: typically O(1) average. 

- Iteration over set: O(n). 

---

## 4. Important Caveats & Practical Implications

- **Amortized vs worst-case**: some operations (like `list.append`) are O(1) amortized — but may occasionally be O(n). 

- **Hash collisions degrade performance**: for `dict` and `set`, pathological cases can degrade average O(1) to O(n).

- **Data structure choice matters**: using a `list` for frequent membership checks or frequent insertions/deletions in the middle often leads to poor performance; `dict` or `set` may be more efficient.  

- **Memory overhead**: built-in types carry additional overhead. For example, `list` stores Python objects (pointers, reference counts), making them more memory-heavy than arrays from lower-level languages or optimized data structures (e.g. from `numpy`).

---

## 5. Summary & Guidelines

- Favor `dict` / `set` when you need fast membership tests, insertions, or lookups.  

- Use `list` when order matters, or when you primarily do appends / index-based access.  

- Beware of operations on lists that require shifting elements (insertions/deletions in middle, membership tests) — these are O(n).  

- For bulk operations or large data volumes, always consider algorithmic complexity: even built-in methods incur costs, and choice of data structure matters.  

- Understanding the cost of built-ins helps anticipate behavior under load and write scalable, performant Python code.

---

### Comparsion about pop() and popleft()

In [1]:
import timeit
from collections import deque

In [4]:
setup_code = """
from collections import deque
N = 100_000
my_list = list(range(N))
my_deque = deque(range(N))
"""

In [5]:
# Test list.pop(0)
list_time = timeit.timeit("my_list.pop(0)", setup=setup_code, number=1000)

# Test deque.popleft()
deque_time = timeit.timeit("my_deque.popleft()", setup=setup_code, number=1000)

print(f"List pop(0): {list_time:.4f} seconds")
print(f"Deque popleft(): {deque_time:.4f} seconds")

List pop(0): 0.0456 seconds
Deque popleft(): 0.0001 seconds
